In [40]:
import pandas as pd
import numpy as np
import wtfutil
import time
import pathlib
import datetime
import functools
import math
import json

In [41]:
FEEAGG_BUCKETS=25

PATH_PREFIX=(datetime.datetime.utcnow()-datetime.timedelta(0)).strftime('data/%Y/%m/%d/') 
#PATH_PREFIX=datetime.datetime.utcnow().strftime('data/2017/12/26/') 
DELTA_DATAPOINTS=120 # 40 dp * 15 sec/dp = 600

COLORS=['#8cff66', '#ff668c', '#668cff']

# load the last DELTA_DATAPOINTS deltas
txpool_delta = pd.concat(wtfutil.pqload_lastn(PATH_PREFIX, 'diff', DELTA_DATAPOINTS)).query('diff == 1')
feeagg_delta = wtfutil.feeagg(txpool_delta, FEEAGG_BUCKETS) * 40 / DELTA_DATAPOINTS

txpool_mpool = pd.concat(wtfutil.pqload_lastn(PATH_PREFIX, 'full', 1).values())
feeagg_mpool = wtfutil.feeagg(txpool_mpool, FEEAGG_BUCKETS)


In [42]:
fn = functools.partial(wtfutil.cellfn_simulate, fa_start=feeagg_mpool, fa_delta=feeagg_delta, mvs=1.0)
df_simulation_large = wtfutil.df_simulation_large

fa_demo = feeagg_mpool + feeagg_delta*10 # more exact since realtime
fn = functools.partial(wtfutil.cellfn_simulate, fa_start=fa_demo, fa_delta=feeagg_delta, mvs=1.0)
r=wtfutil.mp_map_df(fn, df_simulation_large)


data = json.dumps(r.to_dict('split')).replace('NaN', 'null')
with open('mvp/build/data.json', 'w') as f:
    f.write(data)


